# Aeroblade Deep Learning Project

### Import libraries:


In [ ]:
!pip install --user pandas numpy scikit-learn tensorflow matplotlib ipywidgets seaborn scipy
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Normalization
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
from ipywidgets import interact, widgets
import seaborn as sns

### Read simulation data
The data is read from a csv file into a pandas dataframe.
The pandas dataframe is one of the standard input formats for most machine learning frameworks in python.
Columns from the csv file are stored in named columns in the dataframe.
Columns can be accessed by name or by index.
Rows can be accessed by index.

We print the dataframe to see if we imported the csv file correctly.

In [ ]:
file_path = r"k_aeroblade.csv"  # Update this path Nafems/k_aeroblade.csv
print(file_path)
df = pd.read_csv(file_path, delimiter = ';', decimal = ',')
print(df)

### Data Review
seaborn (sns) is used to create plots for data analysis.

#### Histograms
Histograms show the distribution of values for one column in the dataframe.
the code creates a histogram that counts the number of values that fall between 0 and 1, 1 and 2, 2 and 3
Histograms are used to detect imbalances in the dataset.

#### Correlation Matrices
Correlation matrices match every column of the dataframe with each other and calculate the correlation.
**pearson** is used for linear correlations.
**spearman** is used for simple nonlinear correlations.
Parameters that strongly correlate with one another should not be used together as model inputs.
Input parameters which correlate with output parameters should be used as inputs for the model. But weak correlation in the matrix does not mean a parameter should not be used as input, since the actual correlations might be more complex and depend on multiple parameters.

In [ ]:
sns.histplot(df['k10'], bins = [0, 1, 2, 3])
plt.show()

sns.heatmap(df.corr(method='spearman'), annot=True)
plt.show()

#### Data Split
First, we remove the columns we don't want to use from the dataset using drop.
In this case, we split the data manually, and assign the rows systematically. We can do that because we know the data was randomly sampled. If the data was sampled systematically, we would use a randomizer to assign datapoints to the split sets.

In [ ]:
df = df.drop(columns=['k1', 'k3',])
training_data = df.iloc[0:6]
val_data = df.iloc[6:8]
test_data = df.iloc[8:]
print(training_data)
print(val_data)
print(test_data)

#### Model Architecture
This bracket determines the neural networks architecture. <br>
First, the training inputs are selected are selected and stored in a separate variable.
The training inputs are then used to calibrate a **normalization layer** for the neural network. <br>
The normalization layer transforms all incoming data, so that the originally selected input has a mean of 0 and a standard deviation of 1. <br>

Inside **Sequential** we define the model architecture itself. Using Sequential, we can define as many layers as necessary in a simple sequence. <br>
The current model has one predefined **input layer** matching our input data. <br>
It is followed by the **normalization layer**, which we calibrated earlier. <br>
The next three lines are deactivated (turned into comments). To activate them remove the # sign.<br>
**Dense** layers are densely connected feedforward layers. The first number sets the number of neurons for that layer, activation sets the activation function for that layer. <br>
One of the dense layers has a regularizer. Regularizers prevent overfitting, by forcing the optimizer to use small values for the neural networks weights. <br>
The final dense layer uses a linear activation function (the same as no activation function) and serves as the networks output layer. <br>
One of the comments is a **dropout** layer. Dropout layers mitigate overfitting, by randomly deactivating a fraction of the following layers neurons. <br><br>

Inside compile the optimizer and the loss function are set. Metrics use loss functions to evaluate model performance and accuracy, but are not used for training. <br>
Useful optimizers for this case are: 'adam', and 'SGD' (stochastic gradient descent) <br>
Useful loss functions are 'mse' (mean squared error), 'mae' (mean absolute error), 'mape' (mean absolute percentage error)



In [ ]:
training_input = training_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]]

def newmodel(training_input):
    normalization = Normalization()
    normalization.adapt(training_input.to_numpy())
    
    model = Sequential([   
        Input(shape=(training_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]].shape[1],)),  
        normalization,
        #Dense(10, activation='relu'),
        #Dense(10, activation='relu'),
        #Dense(10, activation='relu'),
        #keras.layers.Dropout(0.5),
        Dense(32, activation='relu', kernel_regularizer= keras.regularizers.l1(0.1)),
        Dense(1)  # Output layer for regression (predicting downforce)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mape'])
    return model

model = newmodel(training_input)
print(model.summary())

#### Fitting and Evaluation
This brackets trains the model. First the training and the validation datasets are defined. <br>
The most important parameter here is **epochs**. This is the number of training iterations for the optimizer. Increase epochs if the model does not fit properly. Decrease if it overfits. <br>
Run this bracket multiple times to see random initialization changes the model in each training attempt.

In [ ]:
model = newmodel(training_input)
history = model.fit(
    training_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]], training_data[['k10']],
    validation_data=(val_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]], val_data[['k10']]),
    epochs=800,
    batch_size=6,
    verbose=1
)

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

#### Test Evaluation
This bracket evalutes the model with the selected test data. Check this metric only after settling on a preliminary model architecture to avoid model bias.

In [ ]:
test_loss, test_metrics = model.evaluate(test_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]], test_data['k10'])
print(test_metrics)

#### Prediction
This line of code predicts and prints the k10 value for the selected mesh configuration. This how the ML model is used in practice.

In [ ]:
print(model.predict(np.array([[5,0,0,0,5,0,0,0,0,5]])))

#### Active Sampling
This is a very simple approach to active sampling. We create 10 models of the same architecture and predict the k10 value of 1000 different configurations. We calculate the variance of the predictions to see where the models disagree the most (query-by-comittee). That configuration is what we would simulate next and add to our dataset. <br>
(Please note that this approach to active sampling is simplified to reduce computational time)


The formula for sample variance is:

$$
S^2 = \frac{\sum (x_i - \bar{x})^2}{n - 1}
$$

In [ ]:
# Set the seed for reproducibility
np.random.seed(42)

# Define the number of samples and the shape of each sample
num_samples = 1000
sample_shape = 10

# Generate 1000 random samples with integers between 0 and 5 (inclusive)
np.random.seed(42)
input_data = np.random.randint(0, 6, size=(num_samples, sample_shape))

print(input_data)

models = []
for i in range(10):
    model = newmodel(training_input)
    model.fit(
    training_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]], training_data[['k10']],
    validation_data=(val_data[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',]], val_data[['k10']]),
    epochs=200,
    #batch_size=32,
    verbose=0
)
    models.append(model)

# Generate predictions for each model
all_predictions = []

for model in models:
    # Get the predictions for the 1000 inputs
    predictions = model.predict(input_data)
    all_predictions.append(predictions)

# Convert the list of predictions to a numpy array
# Shape will be (10, 1000, output_shape) if output is multi-dimensional, otherwise (10, 1000)
all_predictions = np.array(all_predictions)

# Calculate variance along the axis of models (axis=0)
# This calculates the variance for each of the 1000 inputs across the 10 models
variance = np.var(all_predictions, axis=0)

# Find the index of the maximum variance
max_variance_index = np.argmax(variance)

# Retrieve the value of the maximum variance
max_variance_value = variance[max_variance_index]

# Retrieve the input vector at this index
input_vector_at_max_variance = input_data[max_variance_index]

#Calculate the mean variance per input
mean_variance = np.mean(variance)
# Print the results
print(f"Mean Configuration Variance: {mean_variance}")
print(f"Median Configuration Variance: {np.median(variance)}")
print(f"Maximum Configuration Variance Value: {max_variance_value}")
print(f"Maximum Variance Configuration: {input_vector_at_max_variance}")

In [ ]:
sns.histplot(variance)
plt.show()

#### Save and Load
The next brackets are used to save a model to a file and load it again in another script.

In [ ]:
model.save('model.keras')

In [ ]:
loaded_model = keras.models.load_model('model.keras')
print(loaded_model.predict(np.array([[0,0,5,3,0,5,0,0,0,5]])))

#### Additional SVM solution

In [ ]:
import pandas as pd
from sklearn.svm import SVR  # Support Vector Regressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
training_input = training_data[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']]
training_target = training_data['k10']
val_input = val_data[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']]
val_target = val_data['k10']
test_input = test_data[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']]
test_target = test_data['k10']

scaler = StandardScaler()
training_input_scaled = scaler.fit_transform(training_input)
val_input_scaled = scaler.transform(val_input)
test_input_scaled = scaler.transform(test_input)

# Train SVM
svm_model = SVR(epsilon=0.1, max_iter=-1)
svm_model.fit(training_input_scaled, training_target)

# Evaluate on validation set
val_predictions = svm_model.predict(val_input_scaled)
val_mse = mean_squared_error(val_target, val_predictions)
print("Validation MSE:", val_mse)

# Evaluate on test set
test_predictions = svm_model.predict(test_input_scaled)
test_mse = mean_squared_error(test_target, test_predictions)
print("Test MSE:", test_mse)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

features = df[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']]


pca_only = PCA()#(n_components=2)

features_pca_only = pca_only.fit_transform(features)


plt.figure(figsize=(8, 6))
plt.scatter(features_pca_only[:, 0], features_pca_only[:, 1], cmap='viridis', s=50, alpha=0.7)
plt.title("PCA Visualization of Features (x1 to x10)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()

n_clusters = 3  # Number of clusters to use
kmeans_for_features = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_labels_for_features = kmeans_for_features.fit_predict(features)


plt.figure(figsize=(8, 6))
plt.scatter(features_pca_only[:, 0], features_pca_only[:, 1], c=kmeans_labels_for_features, cmap='viridis', s=50, alpha=0.7)
plt.scatter(kmeans_for_features.cluster_centers_[:, 0], kmeans_for_features.cluster_centers_[:, 1], c='red', marker='x', s=200, label='Centroids')
plt.title("K-means Clustering with PCA Visualization")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend()
plt.show()